# ai with pytorch for detecting heart disease in humans

## import dependencies

In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

## dataset stuff

### import dataset csv

In [8]:
df = pd.read_csv('dataset.csv')
df = df.sample(frac=1)

#show table
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
69,46,1,3,150,231,0,0,147,0,3.6,2,0.0,3.0,1
158,60,1,4,140,293,0,2,170,0,1.2,2,2.0,7.0,2
243,61,1,1,134,234,0,0,145,0,2.6,2,2.0,3.0,2
164,48,1,3,124,255,1,0,175,0,0.0,1,2.0,3.0,0
22,58,1,2,120,284,0,2,160,0,1.8,2,0.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,60,0,4,150,258,0,2,157,0,2.6,2,2.0,7.0,3
291,55,0,2,132,342,0,0,166,0,1.2,1,0.0,3.0,0
117,35,0,4,138,183,0,0,182,0,1.4,1,0.0,3.0,0
89,51,0,3,130,256,0,2,149,0,0.5,1,0.0,3.0,0


### train test split

In [9]:
ratio = 0.99
total_rows = df.shape[0]
train_size = int(total_rows*ratio)

# Split data into a test dataset and train dataset
train = df[0:train_size]
test = df[train_size:]

#convert to numpy arrays (so its autodiff compatible)
train_data = train.to_numpy()
test_data = test.to_numpy()

#check
print(df)

print(train)
print(test)

print(train_data)
print(test_data)

     age   sex   cp   trestbps   chol   fbs   restecg   thalach   exang  \
69    46     1    3        150    231     0         0       147       0   
158   60     1    4        140    293     0         2       170       0   
243   61     1    1        134    234     0         0       145       0   
164   48     1    3        124    255     1         0       175       0   
22    58     1    2        120    284     0         2       160       0   
..   ...   ...  ...        ...    ...   ...       ...       ...     ...   
97    60     0    4        150    258     0         2       157       0   
291   55     0    2        132    342     0         0       166       0   
117   35     0    4        138    183     0         0       182       0   
89    51     0    3        130    256     0         2       149       0   
119   65     1    4        135    254     0         2       127       0   

      oldpeak   slope   ca  thal   diagnosis  
69        3.6       2  0.0   3.0           1  
158  

## acc. ai stuff

### training

- using pytorch, numpy, and matplotlib (probably:) )
- using a high train test split ratio to get the best results

In [10]:
class SimpleNN(nn.Module): 
  def __init__(self): 
    super(SimpleNN, self).__init__() 
    self.fc1 = nn.Linear(2, 5)   
    self.relu = nn.ReLU()      # Activation function 
    self.fc2 = nn.Linear(5, 1)  

  def forward(self, x): 
    x = self.fc1(x) 
    x = self.relu(x) 
    x = self.fc2(x) 
    return x 


In [11]:
model = SimpleNN() 

# outputs the struct. of the model
print(model) 

SimpleNN(
  (fc1): Linear(in_features=2, out_features=5, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=5, out_features=1, bias=True)
)


In [12]:
# sample data for training
inputs = torch.tensor(train.apply(pd.to_numeric, errors='coerce').fillna(0))

# Extract the diagnosis column and convert to numeric
diagnosis = pd.to_numeric(train.loc[:," diagnosis"], errors='coerce').fillna(0)

# Convert to tensor and reshape to (N, 1)
targets = torch.tensor(diagnosis.values, dtype=torch.float32).view(-1, 1)
#targets = torch.tensor(train['diagnosis'].apply(pd.to_numeric, errors='coerce').fillna(0) * 100, dtype=torch.float32)  # example; replace with your actual targets
targets = targets[:inputs.shape[0]]  # truncate if needed
targets = targets.view(-1, 1)        # ensure shape is (N, 1)

ValueError: could not determine the shape of object type 'DataFrame'

In [ ]:
#calculate mean squared error
criterion = nn.MSELoss() 
optimiser = optim.SGD(model.parameters(), lr=0.01) 

In [ ]:
for epoch in range(1000000):                      # Training for 5 epochs 
  optimiser.zero_grad()                     # Clear previous gradients 
  outputs = model(inputs)                # Forward pass 
  loss = criterion(outputs, targets)  # Calculate loss 
  loss.backward()                             # Backward pass to compute gradients 
  optimiser.step()                             # Update weights 
  print(f'Epoch [{epoch + 1}/1000000], Loss: {loss.item():.4f}') 
outputs

TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame